<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
new home
</font>
</h1>

<p dir=rtl style="direction: ltr; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
This practical tutorial covers exploratory data analysis, preprocessing, and building a deep neural network in Keras to predict pet adoption success.
</font>
</p>

In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [3]:
import keras
print(keras.__version__)

3.4.1


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده
</font>
</h2>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

|ستون|توضیحات|
|:------:|:---:|
|<code>Type</code>| نوع حیوان|
|<code>Age</code>| سن حیوان خانگی (به ماه) هنگام لیست شدن|
|<code>Breed1</code>|نژاد اصلی حیوان خانگی|
|<code>Gender</code>|جنسیت حیوان خانگی|
|<code>Color1</code>|رنگ اصلی حیوان خانگی|
|<code>Color2</code>|رنگ دوم حیوان خانگی (در صورت وجود)|
|<code>MaturitySize</code>|اندازه|
|<code>FurLength</code>|طول خز|
|<code>Vaccinated</code>|وضعیت واکسینه شدن|
|<code>Sterilized</code>|وضعیت انگل‌زدایی شدن|
|<code>Health</code>|وضعیت سلامتی|
|<code>Fee</code>|هزینه|
|<code>Description</code>|توضیحات|
|<code>PhotoAmt</code>|تعداد عکس بارگذاری شده|
|<code>AdoptionSpeed</code>|سرعت پذیرش (توضیح این اعداد در ادامه آمده است)|

<br>

|توضیحات|سرعت پذیرش|
|:------:|:---:|
|حیوان خانگی در همان روزی که لیست شده به سرپرستی پذیرفته شد.|<code>0</code>|
|	حیوان خانگی بین ۱ تا ۷ روز (هفته اول) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>1</code>|
|حیوان خانگی بین ۸ تا ۳۰ روز (ماه اول) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>2</code>|
|حیوان خانگی بین ۳۱ تا ۹۰ روز (ماه دوم و سوم) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>3</code>|
|پس از ۱۰۰ روز لیست شدن، حیوان خانگی به سرپرستی پذیرفته نشد.|<code>4</code>|

</font>
</div>
</center>

In [5]:
train = pd.read_csv('data\petfinder_train.csv')
test = pd.read_csv('data\petfinder_test.csv')

In [6]:
train.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           10537 non-null  object
 1   Age            10537 non-null  int64 
 2   Breed1         10537 non-null  object
 3   Gender         10537 non-null  object
 4   Color1         10537 non-null  object
 5   Color2         10537 non-null  object
 6   MaturitySize   10537 non-null  object
 7   FurLength      10537 non-null  object
 8   Vaccinated     10537 non-null  object
 9   Sterilized     10537 non-null  object
 10  Health         10537 non-null  object
 11  Fee            10537 non-null  int64 
 12  Description    10528 non-null  object
 13  PhotoAmt       10537 non-null  int64 
 14  AdoptionSpeed  10537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.2+ MB


<h2 dir=rtl align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Data Preprocessing</font>
</h2>


In [8]:
train['Target'] = np.where(train['AdoptionSpeed'] < 4, True, False)

In [9]:
train.drop(columns=['AdoptionSpeed', 'Description'], inplace=True)
test.drop(columns=['Description'], inplace=True)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          10537 non-null  object
 1   Age           10537 non-null  int64 
 2   Breed1        10537 non-null  object
 3   Gender        10537 non-null  object
 4   Color1        10537 non-null  object
 5   Color2        10537 non-null  object
 6   MaturitySize  10537 non-null  object
 7   FurLength     10537 non-null  object
 8   Vaccinated    10537 non-null  object
 9   Sterilized    10537 non-null  object
 10  Health        10537 non-null  object
 11  Fee           10537 non-null  int64 
 12  PhotoAmt      10537 non-null  int64 
 13  Target        10537 non-null  bool  
dtypes: bool(1), int64(3), object(10)
memory usage: 1.1+ MB


In [11]:
for column in ["Type", "Breed1", "Gender", "Color1", "Color2", "MaturitySize", "FurLength", "Vaccinated", "Sterilized", "Health"]:
    print("column:", column, "unique_values:", len(train[column].unique()))

column: Type unique_values: 2
column: Breed1 unique_values: 165
column: Gender unique_values: 2
column: Color1 unique_values: 7
column: Color2 unique_values: 7
column: MaturitySize unique_values: 3
column: FurLength unique_values: 3
column: Vaccinated unique_values: 3
column: Sterilized unique_values: 3
column: Health unique_values: 3


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train['MaturitySize'] = le.fit_transform(train['MaturitySize'])
test['MaturitySize'] = le.transform(test['MaturitySize'])

train['FurLength'] = le.fit_transform(train['FurLength'])
test['FurLength'] = le.transform(test['FurLength'])

train['Health'] = le.fit_transform(train['Health'])
test['Health'] = le.transform(test['Health'])

In [ ]:
label_encoder = LabelEncoder()

train['Type'] = label_encoder.fit_transform(train['Type'])
test['Type'] = label_encoder.transform(test['Type'])

train['Gender'] = label_encoder.fit_transform(train['Gender'])
test['Gender'] = label_encoder.transform(test['Gender'])

In [16]:
import category_encoders as ce

binary_encoder = ce.BinaryEncoder()

train_binary_Breed1 = binary_encoder.fit_transform(train['Breed1'])
test_binary_Breed1 = binary_encoder.transform(test['Breed1'])

train_binary_Color1 = binary_encoder.fit_transform(train['Color1'])
test_binary_Color1 = binary_encoder.transform(test['Color1'])

train_binary_Color2 = binary_encoder.fit_transform(train['Color2'])
test_binary_Color2 = binary_encoder.transform(test['Color2'])

train_binary_Vaccinated = binary_encoder.fit_transform(train['Vaccinated'])
test_binary_Vaccinated = binary_encoder.transform(test['Vaccinated'])

train_binary_Sterilized = binary_encoder.fit_transform(train['Sterilized'])
test_binary_Sterilized = binary_encoder.transform(test['Sterilized'])

In [17]:
train = pd.concat([train, train_binary_Breed1, train_binary_Color1, train_binary_Color2, train_binary_Vaccinated, train_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
train.drop(columns = columns, inplace = True)

test = pd.concat([test, test_binary_Breed1, test_binary_Color1, test_binary_Color2, test_binary_Vaccinated, test_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
test.drop(columns = columns, inplace = True)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Type          10537 non-null  int32
 1   Age           10537 non-null  int64
 2   Gender        10537 non-null  int32
 3   MaturitySize  10537 non-null  int32
 4   FurLength     10537 non-null  int32
 5   Health        10537 non-null  int32
 6   Fee           10537 non-null  int64
 7   PhotoAmt      10537 non-null  int64
 8   Target        10537 non-null  bool 
 9   Breed1_0      10537 non-null  int64
 10  Breed1_1      10537 non-null  int64
 11  Breed1_2      10537 non-null  int64
 12  Breed1_3      10537 non-null  int64
 13  Breed1_4      10537 non-null  int64
 14  Breed1_5      10537 non-null  int64
 15  Breed1_6      10537 non-null  int64
 16  Breed1_7      10537 non-null  int64
 17  Color1_0      10537 non-null  int64
 18  Color1_1      10537 non-null  int64
 19  Color1_2      10537 non-n

<h2 dir=rtl align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Data Normalization</font>


In [ ]:
columns = [col for col in train.columns if col != 'Target']

for column in columns:
    mean = train[column].mean() 
    std = train[column].std()     
    train[column] = (train[column] - mean) / std
    
    test[column] = (test[column] - mean) / std

test.head()

,Type,Age,Gender,MaturitySize,FurLength,Health,Fee,PhotoAmt,Breed1_0,Breed1_1,...,Color1_0,Color1_1,Color1_2,Color2_0,Color2_1,Color2_2,Vaccinated_0,Vaccinated_1,Sterilized_0,Sterilized_1
0,0.866680,-0.501042,-0.879677,-0.226574,0.790516,-0.194365,-0.29796,-0.830874,-0.073746,-0.184801,...,-0.521041,-0.861486,0.761850,-0.520158,0.777326,0.637933,-1.151487,0.386892,-0.730867,0.360165
1,-1.153719,0.015102,-0.879677,-0.226574,-0.845318,4.568489,-0.29796,-0.830874,-0.073746,-0.184801,...,-0.521041,1.160675,-1.312469,1.922312,0.777326,-1.567414,0.868360,-2.584453,1.368109,-2.776244
2,-1.153719,-0.449427,-0.879677,1.629890,0.790516,-0.194365,-0.29796,-0.512200,-0.073746,-0.184801,...,-0.521041,-0.861486,0.761850,-0.520158,-1.286339,0.637933,-1.151487,0.386892,-0.730867,0.360165
3,0.866680,-0.501042,-0.879677,-0.226574,-0.845318,-0.194365,-0.29796,0.443820,-0.073746,-0.184801,...,-0.521041,1.160675,0.761850,-0.520158,0.777326,0.637933,0.868360,0.386892,-0.730867,0.360165
4,0.866680,0.015102,-0.879677,-0.226574,-2.481151,-0.194365,-0.29796,-0.512200,-0.073746,-0.184801,...,1.919052,1.160675,-1.312469,-0.520158,0.777326,0.637933,0.868360,-2.584453,1.368109,-2.776244


In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop(columns=['Target'])
y = train['Target']  

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train examples:', len(X_train), len(y_train))
print('Validation examples:', len(X_valid), len(y_valid))
print('Test examples:',  len(test))

Train examples: 9483 9483
Validation examples: 1054 1054
Test examples: 1000


<h2 dir=rtl align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Building the Neural Network</font>
</h2>


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()

input_shape = X_train.shape[1]  
model.add(layers.Input(shape=(input_shape,)))  
model.add(layers.Dense(5000, activation='relu'))  # لایه متراکم با 5000 نورون و تابع فعال‌ساز relu
model.add(layers.Dense(1000, activation='relu'))  # لایه متراکم با 1000 نورون و تابع فعال‌ساز relu
model.add(layers.Dense(500, activation='relu'))   # لایه متراکم با 500 نورون و تابع فعال‌ساز relu

# افزودن لایه خروجی با یک نورون و تابع فعال‌ساز sigmoid (برای پیش‌بینی مقادیر باینری)
model.add(layers.Dense(1, activation='sigmoid'))

In [22]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 5000)           │       135,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1000)           │     5,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 500)            │       500,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,637,001 (21.50 MB)

 Trainable params: 5,637,001 (21.50 MB)

 Non-trainable params: 0 (0.00 B)

<h2 dir=rtl align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Neural Network Training</font>


In [23]:
model.compile(optimizer = 'adam',
              loss = keras.losses.BinaryCrossentropy(from_logits=False),
              metrics = ['accuracy'])

In [24]:
epochs = 10
BATCH_SIZE = 128

# آموزش مدل
history = model.fit(
    X_train, y_train,               # داده‌های آموزشی
    batch_size=BATCH_SIZE,          # اندازه دسته
    epochs=epochs,                  # تعداد دورهای آموزشی
    validation_data=(X_valid, y_valid)  # داده‌های اعتبارسنجی
)

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.7096 - loss: 0.5692 - val_accuracy: 0.7467 - val_loss: 0.5270
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.7524 - loss: 0.5158 - val_accuracy: 0.7524 - val_loss: 0.5176
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.7595 - loss: 0.5006 - val_accuracy: 0.7476 - val_loss: 0.5082
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.7712 - loss: 0.4855 - val_accuracy: 0.7495 - val_loss: 0.4987
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - accuracy: 0.7685 - loss: 0.4824 - val_accuracy: 0.7457 - val_loss: 0.5034
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.7807 - loss: 0.4735 - val_accuracy: 0.7552 - val_loss: 0.5115
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.7772 - loss: 0.4654 - val_accuracy: 0.7514 - val_loss: 0.5261
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.7823 - loss: 0.4585 - val_accuracy: 0.7505 - v


<h2 align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Making Predictions on Test Data & Generating Output</font>
</h2>





<center>
<div dir=ltr style="direction: ltr;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
||<code>Target</code>|
|:----:|:-----:|
|0|<code>True</code>|
|1|<code>True</code>|
|2|<code>False</code>|
|3|<code>False</code>|
|4|<code>True</code>|

</font>
</div>
</center>

In [25]:
test_predictions = model.predict(test)

# تبدیل پیش‌بینی‌ها به مقادیر True/False
# فرض می‌کنیم مدل یک خروجی با احتمال (بین 0 و 1) می‌دهد
# بنابراین از یک آستانه (threshold) برای تبدیل به True/False استفاده می‌کنیم
threshold = 0.5
test_predictions = (test_predictions >= threshold)

# ساخت دیتافریم نهایی برای ارسال
submission = pd.DataFrame({'Target': test_predictions.flatten()})

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [ ]:
import zipfile
import joblib
import numpy as np
import json
import os

if not os.path.exists(os.path.join(os.getcwd(), 'new_home.ipynb')):
    %notebook -e new_home.ipynb

test.to_csv("test.csv")

model_info=[]
for layer in model.layers:
  if layer.__class__.__name__=="Dense":
    model_info.append({
        "name": layer.__class__.__name__,
        "units":layer.units,
        "activation":layer.get_config()["activation"]
    })
  else:
    model_info.append({"name": layer.__class__.__name__})

with open("model_info.json", "w") as f:
    json.dump(model_info, f)


def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)

file_names = ["test.csv",'submission.csv', 'model_info.json', 'new_home.ipynb']
compress(file_names)